# Detección de Mascarillas: CNN Personalizada y Transfer Learning

**Análisis y Comparación de Arquitecturas para la Localización de Objetos**

Este notebook aborda el problema de la detección de mascarillas a través de dos enfoques principales:
1.  **CNN Personalizada:** Se diseña, entrena y evalúa una Red Neuronal Convolucional desde cero.
2.  **Transfer Learning:** Se adaptan dos modelos pre-entrenados (EfficientNet-B0 y Swin Transformer V2) para la misma tarea.

Se explorará el impacto del **aumento de datos** y el ajuste de **hiperparámetros** como el learning rate y el optimizador para mejorar el rendimiento en la clasificación y la regresión del cuadro delimitador.

In [32]:
# ===========================
# CONFIGURACIÓN INICIAL
# ===========================

import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.optim import Optimizer
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchsummary import summary
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
import albumentations as A
import cv2
import os
import os.path as osp
from PIL import Image
import matplotlib.pyplot as plt
import typing as ty
import copy
from functools import reduce

# Ignorar advertencias para una salida más limpia
import warnings
warnings.filterwarnings('ignore')

# Configuración de reproducibilidad y dispositivo
torch.manual_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Usando el dispositivo: {device}')

Usando el dispositivo: cuda


In [33]:
# ===========================
# CARGA Y PREPARACIÓN DE DATOS
# ===========================

# Rutas de los directorios
DATA_DIR = './kaggle/input/aa-iv-2025-ii-object-localization'
IMG_DIR = osp.join(DATA_DIR, "images")

# Carga del archivo de anotaciones
df = pd.read_csv(osp.join(DATA_DIR, "train.csv"))

# Mapeo de clases a IDs numéricos y viceversa
obj2id = {'no-mask': 0, 'mask': 1}
id2obj = {0: 'no-mask', 1: 'mask'}

df["class_id"] = df["class"].map(obj2id)

# Selección de columnas relevantes
df = df[['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'class', 'class_id']].copy()

print("Estructura del DataFrame:")
df.head()

Estructura del DataFrame:


,filename,xmin,ymin,xmax,ymax,class,class_id
0,video_CDC-YOUTUBE_mp4-63_jpg.rf.2f4f64f6ef712f...,315,249,468,374,no-mask,0
1,IMG_4860_mp4-36_jpg.rf.01a053cabddff2cdd19f04e...,257,237,299,264,no-mask,0
2,IMG_1491_mp4-12_jpg.rf.9df64033aebef44b8bb9a6a...,291,245,582,449,mask,1
3,IMG_4861_mp4-64_jpg.rf.74ab6d1da8a1fa9b8fbb576...,231,229,577,420,no-mask,0
4,IMG_9950-1-_mp4-83_jpg.rf.74dca33810c23ba144d8...,107,168,515,469,no-mask,0


In [34]:
# ===========================
# PREPROCESAMIENTO
# ===========================

# Normalización de las coordenadas del Bounding Box
h_real, w_real = 640, 640  # Dimensiones de las imágenes
df[["ymin", "ymax"]] = df[["ymin", "ymax"]].div(h_real, axis=0)
df[["xmin", "xmax"]] = df[["xmin", "xmax"]].div(w_real, axis=0)

# División estratificada en conjuntos de entrenamiento y validación
train_df, val_df = train_test_split(
    df,
    stratify=df['class_id'],
    test_size=0.20, # 20% para validación
    random_state=42
)

print(f"Tamaño del conjunto de entrenamiento: {train_df.shape[0]} muestras")
print(f"Tamaño del conjunto de validación: {val_df.shape[0]} muestras")

Tamaño del conjunto de entrenamiento: 175 muestras
Tamaño del conjunto de validación: 44 muestras


In [35]:
# ===========================
# CLASE DATASET DE PYTORCH (CORREGIDA V2)
# ===========================

class MaskDataset(Dataset):
    """Dataset para cargar las imágenes, bounding boxes y clases."""
    def __init__(self, df: pd.DataFrame, root_dir: str, transform=None, labeled: bool = True):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
        self.labeled = labeled

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Cargar imagen
        img_name = os.path.join(self.root_dir, self.df.filename.iloc[idx])
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        sample = {'image': image}

        if self.labeled:
            bbox = self.df.iloc[idx, 1:5].values.astype('float')
            class_id = self.df.class_id.iloc[idx]
            sample.update({'bbox': np.array([bbox]), 'class_id': np.array([class_id])})

        if self.transform:
            transform_args = {'image': sample['image']}
            if self.labeled:
                transform_args['bboxes'] = sample['bbox']
                transform_args['category_ids'] = [sample['class_id'].item()]
            
            transformed = self.transform(**transform_args)
            
            sample['image'] = transformed['image']
            if self.labeled:
                # === INICIO DE LA CORRECCIÓN ===
                # Comprobamos la longitud del contenedor para ver si hay bboxes,
                # en lugar de evaluar la "veracidad" del array.
                if len(transformed['bboxes']) > 0:
                    # Albumentations devuelve una tupla por bbox, tomamos solo las coordenadas.
                    bbox_coords = transformed['bboxes'][0][:4]
                    sample['bbox'] = torch.tensor(bbox_coords, dtype=torch.float32)
                else:
                    # Si el aumento de datos eliminó el bbox, creamos un tensor vacío.
                    sample['bbox'] = torch.zeros(4, dtype=torch.float32)
                # === FIN DE LA CORRECCIÓN ===
        
        # Convertir imagen a tensor y permutar dimensiones (H,W,C) -> (C,H,W)
        sample['image'] = torch.from_numpy(sample['image'].transpose((2, 0, 1))).float()
        
        return sample

### Aumento de Datos (Data Augmentation)

Se definen tres niveles de aumento de datos para comparar su impacto:

1.  **Aumento Ligero (`light_augmentations`):**
    *   `HorizontalFlip`: La técnica más común y segura. Refleja la imagen horizontalmente, lo cual es una variación muy probable en escenarios reales (personas vistas de izquierda o derecha). Ayuda al modelo a ser invariante a la orientación.

2.  **Aumento Medio (`medium_augmentations`):**
    *   Incluye el `HorizontalFlip`.
    *   `RandomBrightnessContrast`: Simula diferentes condiciones de iluminación. Es útil porque las fotos pueden ser tomadas en interiores, exteriores, con o sin flash. Entrena al modelo para que no dependa del brillo o contraste específico de la imagen.
    *   `Rotate`: Introduce pequeñas rotaciones. Las cabezas de las personas no siempre están perfectamente verticales. Esto ayuda al modelo a reconocer objetos aunque estén ligeramente inclinados.

3.  **Aumento Pesado (`heavy_augmentations`):**
    *   Incluye las transformaciones anteriores.
    *   `Blur`: Simula imágenes ligeramente desenfocadas o con movimiento, lo que puede ocurrir con cámaras de baja calidad o movimiento rápido.
    *   `CoarseDropout`: Elimina regiones rectangulares de la imagen, forzando al modelo a aprender de características parciales del objeto y a no depender de una sola región específica (como los ojos o la nariz).

In [36]:
# ===========================
# PIPELINES DE AUMENTO DE DATOS
# ===========================
IMG_SIZE = 256 # Tamaño de entrada para los modelos

# Parámetros comunes para los bounding boxes en Albumentations
BBOX_PARAMS = A.BboxParams(format='albumentations', label_fields=['category_ids'])

# 1. Aumento Ligero
light_augmentations = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
], bbox_params=BBOX_PARAMS)

# 2. Aumento Medio
medium_augmentations = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=15, p=0.4),
], bbox_params=BBOX_PARAMS)

# 3. Aumento Pesado
heavy_augmentations = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.4),
    A.Rotate(limit=20, p=0.5),
    A.Blur(blur_limit=3, p=0.2),
    A.CoarseDropout(max_holes=8, max_height=16, max_width=16, p=0.3),
], bbox_params=BBOX_PARAMS)

# Transformaciones solo para validación (sin aumento)
val_transforms = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
], bbox_params=BBOX_PARAMS)

## Definición de Modelos

### 1. Backbone: CNN Personalizada
Se crea una CNN simple desde cero que servirá como extractor de características. Consta de tres bloques convolucionales, cada uno con una capa `Conv2d`, `ReLU` para la activación y `MaxPool2d` para reducir las dimensiones espaciales. Finalmente, un `AdaptiveAvgPool2d` asegura que la salida tenga un tamaño fijo antes de pasar a las cabezas.

### 2. Cabezas de Clasificación y Regresión
La clase `Model` es un módulo genérico que integra un `backbone` con dos cabezas:
- **Cabeza de Clasificación (`cls_head`):** Una red neuronal simple (MLP) que toma las características del backbone y predice la probabilidad de cada clase (`mask` o `no-mask`).
- **Cabeza de Regresión (`reg_head`):** Otro MLP que predice las 4 coordenadas del bounding box (`xmin`, `ymin`, `xmax`, `ymax`).

Esta estructura modular permite intercambiar fácilmente el backbone (CNN personalizada, EfficientNet, Swin Transformer).

### 3. Backbones para Transfer Learning
- **EfficientNet-B0:** Un modelo ligero y eficiente, conocido por su buen equilibrio entre precisión y costo computacional.
- **Swin Transformer V2:** Un modelo basado en la arquitectura Transformer, que ha demostrado un rendimiento excelente en tareas de visión por computadora al capturar dependencias a larga distancia.

In [37]:
# ===============================================
# 1. BACKBONE: CNN PERSONALIZADA
# ===============================================
class CustomCNN(nn.Module):
    def __init__(self, output_features_dim=256):
        super().__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # 256 -> 128
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # 128 -> 64
        )
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # 64 -> 32
        )
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        self.flatten = nn.Flatten()
        self.final_layer = nn.Linear(128, output_features_dim)

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.final_layer(x)
        return x

# ===============================================
# 2. CABEZAS DE CLASIFICACIÓN Y REGRESIÓN
# ===============================================
class Model(nn.Module):
    def __init__(self, backbone: nn.Module, backbone_out_features: int, n_classes: int = 2):
        super().__init__()
        self.backbone = backbone

        # Cabeza de Regresión
        self.reg_head = nn.Sequential(
            nn.Linear(backbone_out_features, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 4) # 4 coordenadas del bbox
        )

        # Cabeza de Clasificación
        self.cls_head = nn.Sequential(
            nn.Linear(backbone_out_features, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, n_classes) # 2 clases
        )

    def forward(self, x: Tensor) -> ty.Dict[str, Tensor]:
        features = self.backbone(x)
        pred_bbox = self.reg_head(features)
        cls_logits = self.cls_head(features)
        return {'bbox': pred_bbox, 'class_id': cls_logits}

# ===============================================
# 3. BACKBONES PRE-ENTRENADOS
# ===============================================
class EffNetFeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        m = torchvision.models.efficientnet_b0(weights='DEFAULT')
        self.features = m.features
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.flatten(x)
        return x

class SwinV2FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        m = torchvision.models.swin_v2_b(weights='DEFAULT')
        self.features = m.features
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.features(x)
        x = x.permute(0, 3, 1, 2)
        x = self.pool(x)
        x = self.flatten(x)
        return x

In [38]:
# ===========================
# FUNCIONES AUXILIARES
# ===========================

def iou(y_true: Tensor, y_pred: Tensor):
    """Calcula la métrica IoU (Intersection over Union)."""
    # torchvision.ops.box_iou espera [N, 4] y [M, 4]
    if y_true.dim() == 3: y_true = y_true.squeeze(1)
    if y_pred.dim() == 3: y_pred = y_pred.squeeze(1)
    pairwise_iou = torchvision.ops.box_iou(y_true, y_pred)
    # Tomamos la diagonal, asumiendo una correspondencia 1 a 1
    result = torch.diag(pairwise_iou).mean()
    return result

def accuracy(y_true: Tensor, y_pred: Tensor):
    """Calcula la métrica de Accuracy para clasificación."""
    y_true = y_true.squeeze().long()
    pred = torch.argmax(y_pred, dim=-1)
    return (pred == y_true).float().mean()

def loss_fn(y_true, y_preds, alpha=0.5):
    """Función de pérdida combinada para clasificación y regresión."""
    # Pérdida de clasificación (Cross Entropy)
    cls_y_true = y_true['class_id'].squeeze().long()
    cls_loss = F.cross_entropy(y_preds['class_id'], cls_y_true)

    # Pérdida de regresión (Smooth L1 Loss es más robusta a outliers que MSE)
    reg_loss = F.smooth_l1_loss(y_preds['bbox'], y_true['bbox'])

    total_loss = (1 - alpha) * cls_loss + alpha * reg_loss
    return {'loss': total_loss, 'reg_loss': reg_loss, 'cls_loss': cls_loss}

def train_one_epoch(model, dataloader, optimizer, device):
    """Bucle de entrenamiento para una época."""
    model.train()
    total_loss = 0
    pbar = tqdm(dataloader, desc="Entrenando")
    for batch in pbar:
        optimizer.zero_grad()
        # Mover datos al dispositivo
        for key in ['image', 'bbox', 'class_id']:
            batch[key] = batch[key].to(device)

        preds = model(batch['image'])
        losses = loss_fn(batch, preds)
        losses['loss'].backward()
        optimizer.step()

        total_loss += losses['loss'].item()
        pbar.set_postfix(loss=total_loss/len(pbar))

def evaluate(model, dataloader, device):
    """Bucle de evaluación."""
    model.eval()
    total_cls_loss, total_reg_loss = 0, 0
    all_acc, all_iou = [], []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluando"):
            for key in ['image', 'bbox', 'class_id']:
                batch[key] = batch[key].to(device)

            preds = model(batch['image'])
            losses = loss_fn(batch, preds)
            total_cls_loss += losses['cls_loss'].item()
            total_reg_loss += losses['reg_loss'].item()

            all_acc.append(accuracy(batch['class_id'], preds['class_id']).cpu())
            all_iou.append(iou(batch['bbox'], preds['bbox']).cpu())

    avg_acc = np.mean(all_acc)
    avg_iou = np.mean(all_iou)
    print(f"Resultados de validación -> Accuracy: {avg_acc:.4f}, IoU: {avg_iou:.4f}")
    return avg_iou

## Experimento 1: CNN Personalizada con Diferentes Aumentos de Datos

En este experimento, entrenaremos el modelo con el backbone de CNN personalizada desde cero. El objetivo es analizar cómo los diferentes niveles de aumento de datos (`light`, `medium`, `heavy`) afectan el rendimiento del modelo en ambas tareas (clasificación y regresión).

In [39]:
# ================================================================
# EXPERIMENTO 1: CNN PERSONALIZADA + COMPARACIÓN DE AUMENTO DE DATOS
# ================================================================
EPOCHS = 5
LEARNING_RATE = 0.001

augmentation_pipelines = {
    "light": light_augmentations,
    "medium": medium_augmentations,
    "heavy": heavy_augmentations
}

best_custom_cnn_model = None
best_custom_cnn_iou = -1

for name, aug_pipeline in augmentation_pipelines.items():
    print(f"\n--- Entrenando con aumento de datos: {name.upper()} ---")

    # 1. Crear Datasets y DataLoaders
    # Aseguramos que las variables se creen de nuevo en cada iteración
    train_dataset_custom = MaskDataset(train_df, root_dir=IMG_DIR, transform=aug_pipeline)
    val_dataset_custom = MaskDataset(val_df, root_dir=IMG_DIR, transform=val_transforms)

    train_loader_custom = DataLoader(train_dataset_custom, batch_size=32, shuffle=True)
    val_loader_custom = DataLoader(val_dataset_custom, batch_size=32, shuffle=False)

    # 2. Instanciar el modelo (siempre uno nuevo)
    backbone_custom = CustomCNN(output_features_dim=128).to(device)
    model_custom = Model(backbone=backbone_custom, backbone_out_features=128).to(device)
    optimizer_custom = torch.optim.Adam(model_custom.parameters(), lr=LEARNING_RATE)

    # 3. Bucle de entrenamiento
    for epoch in range(EPOCHS):
        print(f"Época {epoch + 1}/{EPOCHS}")
        train_one_epoch(model_custom, train_loader_custom, optimizer_custom, device)
        current_iou = evaluate(model_custom, val_loader_custom, device)

        # Guardar el mejor modelo de este experimento
        if current_iou > best_custom_cnn_iou:
            best_custom_cnn_iou = current_iou
            best_custom_cnn_model = copy.deepcopy(model_custom.state_dict())
            print(f"Nuevo mejor modelo de CNN personalizada con IoU: {best_custom_cnn_iou:.4f}")

# Guardar el mejor modelo de la CNN personalizada en disco
torch.save(best_custom_cnn_model, 'custom_cnn_best.pth')
print("\nMejor modelo de CNN personalizada guardado.")


--- Entrenando con aumento de datos: LIGHT ---
Época 1/5


Entrenando:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluando: 100%|██████████| 2/2 [00:00<00:00,  3.11it/s]


Resultados de validación -> Accuracy: 0.5781, IoU: 0.1434
Nuevo mejor modelo de CNN personalizada con IoU: 0.1434
Época 2/5


Evaluando: 100%|██████████| 2/2 [00:00<00:00,  3.31it/s]


...

Resultados de validación -> Accuracy: 0.5781, IoU: 0.2365
Época 4/5


Evaluando: 100%|██████████| 2/2 [00:00<00:00,  3.46it/s]


Resultados de validación -> Accuracy: 0.5781, IoU: 0.1921
Época 5/5


Evaluando: 100%|██████████| 2/2 [00:00<00:00,  3.44it/s]

Resultados de validación -> Accuracy: 0.5781, IoU: 0.2411

Mejor modelo de CNN personalizada guardado.


## Experimento 2: Transfer Learning

Ahora, utilizaremos modelos pre-entrenados como backbones, conectando nuestras cabezas de clasificación y regresión.

### 2.1 EfficientNet-B0: Ajuste del Learning Rate
Probaremos tres tasas de aprendizaje (`1e-3`, `1e-4`, `1e-5`) para encontrar la que mejor se adapte a la tarea de fine-tuning con EfficientNet. Un learning rate más bajo suele ser preferible para no destruir las características aprendidas por el modelo pre-entrenado.

### 2.2 Swin Transformer V2: Ajuste del Optimizador y Weight Decay
Para el Swin Transformer, compararemos dos optimizadores:
- **Adam:** Un optimizador estándar y robusto.
- **AdamW:** Una variante de Adam que desacopla la regularización de `weight_decay` de la actualización del gradiente, lo que a menudo conduce a una mejor generalización.
Además, probaremos dos valores de `weight_decay` (`1e-4` y `1e-2`) para analizar el efecto de la regularización L2.

In [ ]:
# ================================================================
# EXPERIMENTO 2.1: EFFICIENTNET-B0 + AJUSTE DE LEARNING RATE
# ================================================================
EPOCHS = 10
learning_rates = [1e-3, 1e-4, 1e-5]

best_effnet_model = None
best_effnet_iou = -1

train_dataset_tl = MaskDataset(train_df, root_dir=IMG_DIR, transform=medium_augmentations)
val_dataset_tl = MaskDataset(val_df, root_dir=IMG_DIR, transform=val_transforms)
train_loader_tl = DataLoader(train_dataset_tl, batch_size=16, shuffle=True)
val_loader_tl = DataLoader(val_dataset_tl, batch_size=16, shuffle=False)

for lr in learning_rates:
    print(f"\n--- Entrenando EfficientNet-B0 con LR: {lr} ---")

    backbone_effnet = EffNetFeatureExtractor().to(device)
    model_effnet = Model(backbone=backbone_effnet, backbone_out_features=1280).to(device)
    optimizer_effnet = torch.optim.Adam(model_effnet.parameters(), lr=lr)

    for epoch in range(EPOCHS):
        print(f"Época {epoch + 1}/{EPOCHS}")
        train_one_epoch(model_effnet, train_loader_tl, optimizer_effnet, device)
        current_iou = evaluate(model_effnet, val_loader_tl, device)

        if current_iou > best_effnet_iou:
            best_effnet_iou = current_iou
            best_effnet_model = copy.deepcopy(model_effnet.state_dict())
            print(f"Nuevo mejor modelo EfficientNet con IoU: {best_effnet_iou:.4f}")

torch.save(best_effnet_model, 'efficientnet_best.pth')
print("\nMejor modelo EfficientNet guardado.")

# ================================================================
# EXPERIMENTO 2.2: SWIN TRANSFORMER + AJUSTE DE OPTIMIZADOR
# ================================================================
optimizers_config = [
    {"name": "Adam", "optim": torch.optim.Adam, "wd": 1e-4},
    {"name": "AdamW", "optim": torch.optim.AdamW, "wd": 1e-4},
    {"name": "AdamW_high_wd", "optim": torch.optim.AdamW, "wd": 1e-2},
]

best_swin_model = None
best_swin_iou = -1
LEARNING_RATE_SWIN = 1e-5

for config in optimizers_config:
    print(f"\n--- Entrenando Swin Transformer con Optimizador: {config['name']} (wd={config['wd']}) ---")

    backbone_swin = SwinV2FeatureExtractor().to(device)
    model_swin = Model(backbone=backbone_swin, backbone_out_features=1024).to(device) # Swin-V2-B tiene 1024 features
    optimizer_swin = config["optim"](model_swin.parameters(), lr=LEARNING_RATE_SWIN, weight_decay=config["wd"])

    for epoch in range(EPOCHS):
        print(f"Época {epoch + 1}/{EPOCHS}")
        train_one_epoch(model_swin, train_loader_tl, optimizer_swin, device)
        current_iou = evaluate(model_swin, val_loader_tl, device)

        if current_iou > best_swin_iou:
            best_swin_iou = current_iou
            best_swin_model = copy.deepcopy(model_swin.state_dict())
            print(f"Nuevo mejor modelo Swin Transformer con IoU: {best_swin_iou:.4f}")

torch.save(best_swin_model, 'swin_transformer_best.pth')
print("\nMejor modelo Swin Transformer guardado.")


--- Entrenando EfficientNet-B0 con LR: 0.001 ---
Época 1/3


Evaluando: 100%|██████████| 3/3 [00:00<00:00,  4.00it/s]


Resultados de validación -> Accuracy: 0.7708, IoU: 0.0841
Nuevo mejor modelo EfficientNet con IoU: 0.0841
Época 2/3


Evaluando: 100%|██████████| 3/3 [00:00<00:00,  4.01it/s]


Resultados de validación -> Accuracy: 0.9306, IoU: 0.1631
Nuevo mejor modelo EfficientNet con IoU: 0.1631
Época 3/3


...

Resultados de validación -> Accuracy: 0.6042, IoU: 0.0000
Época 2/3


Evaluando: 100%|██████████| 3/3 [00:18<00:00,  6.12s/it]


Resultados de validación -> Accuracy: 0.6042, IoU: 0.0287
Época 3/3


Evaluando: 100%|██████████| 3/3 [00:18<00:00,  6.13s/it]


Resultados de validación -> Accuracy: 0.6042, IoU: 0.1335
Nuevo mejor modelo Swin Transformer con IoU: 0.1335

Mejor modelo Swin Transformer guardado.


## Generación del Archivo de Submission

Finalmente, cargamos el mejor modelo obtenido de todos los experimentos (el que haya alcanzado el mayor IoU en validación) y lo utilizamos para generar las predicciones sobre el conjunto de test. Las predicciones se guardan en un archivo `submission.csv` con el formato requerido por la competencia.

In [47]:
# ===============================================
# GENERACIÓN DE SUBMISSION (CORREGIDA)
# ===============================================

test_df = pd.read_csv(osp.join(DATA_DIR, "test.csv"))

test_transforms = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE)
])

test_dataset = MaskDataset(test_df, root_dir=IMG_DIR, transform=test_transforms, labeled=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

print("Cargando el mejor modelo para la inferencia...")
best_backbone = EffNetFeatureExtractor().to(device) # Cambiar si otro modelo fue el mejor
best_model = Model(backbone=best_backbone, backbone_out_features=1280).to(device)
best_model.load_state_dict(torch.load('efficientnet_best.pth'))
best_model.eval()

filenames, class_preds, xmins, ymins, xmaxs, ymaxs = [], [], [], [], [], []

with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader, desc="Generando predicciones")):
        image = batch['image'].to(device)
        preds = best_model(image)

        class_id = torch.argmax(preds['class_id'], dim=1).cpu().item()
        bbox = preds['bbox'].cpu().numpy()[0]

        xmin = bbox[0] * w_real
        ymin = bbox[1] * h_real
        xmax = bbox[2] * w_real
        ymax = bbox[3] * h_real

        filenames.append(test_df.filename.iloc[i])
        class_preds.append(id2obj[class_id])
        xmins.append(xmin)
        ymins.append(ymin)
        xmaxs.append(xmax)
        ymaxs.append(ymax)

submission_df = pd.DataFrame({
    'filename': filenames,
    'class': class_preds,
    'xmin': xmins,
    'ymin': ymins,
    'xmax': xmaxs,
    'ymax': ymaxs
})

submission_df = submission_df.set_index('filename')

submission_df.to_csv('submission.csv')

print("\nArchivo 'submission.csv' generado con éxito.")
submission_df.head()

Cargando el mejor modelo para la inferencia...


Generando predicciones: 100%|██████████| 55/55 [00:01<00:00, 48.76it/s]


Archivo 'submission.csv' generado con éxito.


,class,xmin,ymin,xmax,ymax
filename,,,,,
IMG_4861_mp4-50_jpg.rf.7173e37ed9f62f8939af82323289faf2.jpg,no-mask,158.032364,132.117340,420.171753,316.990112
video_CDC-YOUTUBE_mp4-58_jpg.rf.370d5f316397477da0ff4f44799b1da9.jpg,mask,184.373398,70.928772,301.979736,272.172791
video_CDC-YOUTUBE_mp4-57_jpg.rf.de4856b9a314980e4113335576f453a8.jpg,mask,253.181732,66.167816,386.809326,385.631744
IMG_3102_mp4-0_jpg.rf.6a18575fb4bf7f69cc9006b9a5f34e08.jpg,no-mask,281.951721,170.056580,693.483337,630.411255
IMG_3094_mp4-34_jpg.rf.11eecb9601680286dc8338d5e8b9acb2.jpg,no-mask,158.604919,256.652893,711.819397,630.828247
